Use gridsearch to see if there are more optimal lightGBM parameters we should be using.

We'll use the features that a have already been generated for our current best experiment (third sentinel + land cover features, trained with folds). Compare the results to that [best experiment](https://docs.google.com/presentation/d/1zWrSMSivxylx_iH_aOapJfyziRsDuyuXOELduOn6x3c/edit#slide=id.g278eb39bdd6_0_43)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yaml

from cloudpathlib import AnyPath
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import GridSearchCV

from cyano.data.utils import add_unique_identifier
from cyano.experiment.experiment import ExperimentConfig
from cyano.pipeline import CyanoModelPipeline
from cyano.settings import RANDOM_STATE

In [3]:
tmp_dir = AnyPath("tmp_dir")
tmp_dir.mkdir(exist_ok=True)

In [4]:
experiment_dir = AnyPath(
    "s3://drivendata-competition-nasa-cyanobacteria/experiments/results/third_sentinel_with_folds"
)

In [5]:
train_features = pd.read_csv(experiment_dir / "features_train.csv", index_col=0)
train_features.head()

,AOT_mean,AOT_min,AOT_max,AOT_range,B01_mean,B01_min,B01_max,B01_range,B02_mean,B02_min,...,WVP_min,WVP_max,WVP_range,NDVI_B04,NDVI_B05,NDVI_B06,NDVI_B07,month,days_before_sample,land_cover
sample_id,,,,,,,,,,,,,,,,,,,,,
0001cfa683171fe80161cdbe6a090c94,102.0,102.0,102.0,0.0,319.843750,144.0,1083.0,939.0,441.956349,146.0,...,277.0,1413.0,1136.0,0.581754,0.418335,0.114064,0.033551,2.0,15.0,11
0001cfa683171fe80161cdbe6a090c94,132.0,132.0,132.0,0.0,435.984375,229.0,1055.0,826.0,503.288549,154.0,...,167.0,828.0,661.0,0.574375,0.418914,0.120972,0.042572,3.0,8.0,11
0001cfa683171fe80161cdbe6a090c94,204.0,204.0,204.0,0.0,902.640625,703.0,1661.0,958.0,938.553288,672.0,...,931.0,931.0,0.0,0.455419,0.329302,0.097910,0.027511,3.0,5.0,11
0001cfa683171fe80161cdbe6a090c94,102.0,102.0,102.0,0.0,518.484375,316.0,1554.0,1238.0,618.096939,277.0,...,574.0,1436.0,862.0,0.548216,0.391690,0.115218,0.032818,2.0,13.0,11
0001cfa683171fe80161cdbe6a090c94,102.0,102.0,102.0,0.0,318.218750,104.0,1314.0,1210.0,485.587302,162.0,...,340.0,1114.0,774.0,0.582335,0.425137,0.129981,0.039196,2.0,10.0,11


In [6]:
# Load train labels
train = pd.read_csv(
    AnyPath(
        "s3://drivendata-competition-nasa-cyanobacteria/experiments/splits/competition/train.csv"
    )
)
train = add_unique_identifier(train)
train.shape

(17060, 10)

In [7]:
train.head(2)

,uid,data_provider,region,latitude,longitude,date,density_cells_per_ml,severity,distance_to_water_m,log_density
sample_id,,,,,,,,,,
d7ebbce63c7d1498cc627a1e77f6061c,aabm,Indiana State Department of Health,midwest,39.080319,-86.430867,2018-05-14,585.0,1,0.0,6.373320
0856c3740614b5ee606f82d6c3a215a0,aacd,N.C. Division of Water Resources N.C. Departme...,south,35.875083,-78.878434,2020-11-19,290.0,1,514.0,5.673323


In [8]:
train.loc[train_features.index].log_density

sample_id
0001cfa683171fe80161cdbe6a090c94    15.800642
0001cfa683171fe80161cdbe6a090c94    15.800642
0001cfa683171fe80161cdbe6a090c94    15.800642
0001cfa683171fe80161cdbe6a090c94    15.800642
0001cfa683171fe80161cdbe6a090c94    15.800642
                                      ...    
fff76b0c751a22eda5a33f3f0d7fda98    10.338188
fff948d10c4ef9e03fddb358140b2755     9.275660
fff948d10c4ef9e03fddb358140b2755     9.275660
fff948d10c4ef9e03fddb358140b2755     9.275660
fff948d10c4ef9e03fddb358140b2755     9.275660
Name: log_density, Length: 43409, dtype: float64

Try params from each of the winners

In [27]:
param_grid = {
    'max_depth': [-1, 8],
    'num_leaves': [31],
    'learning_rate': [0.005, 0.1],
    'bagging_fraction': [0.3, 1.0],
    'feature_fraction': [0.3, 1.0],
    'min_split_gain': [0.0, 0.1],
    'n_estimators': [100, 1000, 470], # same as num_boost_round
}

Note that this is slightly different than our process because we use LGB.Booster, which we cannot input to the GridSearch. With our grid search, we are not using a valid set or early stopping.

In [28]:
lgb_model = lgb.LGBMModel(
    objective='regression', metric='rmse'
)

In [29]:
grid_search = GridSearchCV(
    estimator = lgb_model,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    scoring='neg_root_mean_squared_error'
)

Note that grid search CV always tries to maximize the score, so root mean squared error has to be negative

In [30]:
%%time
grid_search.fit(
    train_features,
    train.loc[train_features.index].log_density
)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.038116 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15437
[LightGBM] [Info] Number of data points in the train set: 34727, number of used features: 67
[LightGBM] [Info] Start training from score 10.861643
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008782 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 15436
[LightGBM] [Info] Number of data points in the train set: 34727, number of used features: 67
[LightGBM] [Info] Start training fro

GridSearchCV(cv=5, estimator=LGBMModel(metric='rmse', objective='regression'),
             n_jobs=-1,
             param_grid={'bagging_fraction': [0.3, 1.0],
                         'feature_fraction': [0.3, 1.0],
                         'learning_rate': [0.005, 0.1], 'max_depth': [-1, 8],
                         'min_split_gain': [0.0, 0.1],
                         'n_estimators': [100, 1000, 470], 'num_leaves': [31]},
             scoring='neg_root_mean_squared_error')

If we don't specify 'scoring' in `grid_search`, I think score is the `lgb_model.score` method, which is R^2. Unclear whether specifying `metric="rmse"` makes the score that is returned RMSE.

In [33]:
grid_search.best_estimator_.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.1,
 'n_estimators': 1000,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': 'regression',
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'metric': 'rmse',
 'bagging_fraction': 0.3,
 'feature_fraction': 1.0}

In [ ]:
# is the best model for each `n_estimators` the same?

In [35]:
n_est = 1000

In [ ]:
for n_est in results.param_n_estimators.unique():
    print(f'Best params for {n_est} estiamtors 

In [40]:
(results[results.param_n_estimators == n_est]
 .sort_values(by='mean_test_score', ascending=False)
 .iloc[0]['params']
)

{'bagging_fraction': 0.3,
 'feature_fraction': 1.0,
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_split_gain': 0.1,
 'n_estimators': 1000,
 'num_leaves': 31}

[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015857 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15437
[LightGBM] [Info] Number of data points in the train set: 34727, number of used features: 67
[LightGBM] [Info] Start training from score 10.861643
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=1.0, cols

In [43]:
results.to_csv('grid_search_results.csv', index=False)

In [36]:
results = pd.DataFrame(grid_search.cv_results_).sort_values(by='mean_test_score', ascending=False)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bagging_fraction,param_feature_fraction,param_learning_rate,param_max_depth,param_min_split_gain,param_n_estimators,param_num_leaves,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
40,34.928041,2.312749,0.222968,0.010105,0.3,1.0,0.1,-1,0.1,1000,31,"{'bagging_fraction': 0.3, 'feature_fraction': ...",-2.189238,-2.191763,-2.120085,-2.085720,-2.054540,-2.128269,0.054885,1
88,37.013076,0.357481,0.267185,0.048497,1.0,1.0,0.1,-1,0.1,1000,31,"{'bagging_fraction': 1.0, 'feature_fraction': ...",-2.189238,-2.191763,-2.120085,-2.085720,-2.054540,-2.128269,0.054885,1
85,36.701472,0.869560,0.221529,0.009565,1.0,1.0,0.1,-1,0.0,1000,31,"{'bagging_fraction': 1.0, 'feature_fraction': ...",-2.195485,-2.189796,-2.117696,-2.088179,-2.053486,-2.128928,0.055880,3
37,33.746868,0.846992,0.225704,0.011635,0.3,1.0,0.1,-1,0.0,1000,31,"{'bagging_fraction': 0.3, 'feature_fraction': ...",-2.195485,-2.189796,-2.117696,-2.088179,-2.053486,-2.128928,0.055880,3
61,19.513581,2.004610,0.190165,0.011235,1.0,0.3,0.1,-1,0.0,1000,31,"{'bagging_fraction': 1.0, 'feature_fraction': ...",-2.216099,-2.179904,-2.127290,-2.073644,-2.051967,-2.129781,0.061929,5
